## Imports

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
from sklearn import preprocessing

## load data

In [2]:
data=pd.read_csv('../data/AdSmartABdata.csv')

## Data visualization

In [3]:
data.head(5)

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,Chrome Mobile,0,0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,6,Chrome Mobile,0,0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1
3,00187412-2932-4542-a8ef-3633901c98d9,control,2020-07-03,15,Samsung SM-A705FN,6,Facebook,0,0
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0


In [4]:
# umber of unique values for aech column
for col in data.columns: print(col,':', data[col].nunique(), 'labels.')

auction_id : 8077 labels.
experiment : 2 labels.
date : 8 labels.
hour : 24 labels.
device_make : 270 labels.
platform_os : 3 labels.
browser : 15 labels.
yes : 2 labels.
no : 2 labels.


### Feature engineering

In [5]:
data['clicked'] = data['yes']+data['no']
data = data[data['clicked'] == 1]

In [6]:
c_data = data[['experiment','date', 'hour', 'device_make', 'platform_os', 'yes']]

In [7]:
X= c_data[['experiment','date', 'hour', 'device_make', 'platform_os']]
Y = c_data[['yes']].values
X.shape, Y.shape

((1243, 5), (1243, 1))

In [9]:
pro = preprocessing.LabelEncoder()
X['experiment'] = pro.fit_transform(X['experiment'])
X['device_make'] = pro.fit_transform(X['device_make'])
X['date'] = pro.fit_transform(X['date'])
X.sample(5)

<ipython-input-9-15b716e47068>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['experiment'] = pro.fit_transform(X['experiment'])
<ipython-input-9-15b716e47068>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['device_make'] = pro.fit_transform(X['device_make'])
<ipython-input-9-15b716e47068>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.o

,experiment,date,hour,device_make,platform_os
4361,1,6,18,81,6
653,0,0,15,13,6
6238,1,0,3,13,6
4985,0,2,21,13,6
6289,1,5,20,79,6
